In [ ]:
dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [ ]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 3 - 3.2 Segmentação de Politica Antigos

### Geração de Massa

In [ ]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'payloadHomol.intermediarias.segmentacaoNegocio':["Varejo"],
                             'solicitante.flagFuncionario':[0],
                             'payloadHomol.intermediarias.clienteFoPag':[0],
                             'payloadHomol.intermediarias.principalidadeAll':[0],
                             'solicitante.flagMAT':[0],
                             'solicitante.rendaLiquidaPicpay':[10000]
                            })

In [ ]:
#Criação das massas com casos forçados

massa_SegmentacaoPoliticaAntigos = sherlock.Gera_Massa(base_entrada)
massa_SegmentacaoPoliticaAntigos.gera_cenario('payloadHomol.intermediarias.segmentacaoNegocio',["Varejo+","Varejo","Alta renda investidor","Alta renda"])
massa_SegmentacaoPoliticaAntigos.gera_cenario('solicitante.flagFuncionario',[0,1])
massa_SegmentacaoPoliticaAntigos.gera_cenario('payloadHomol.intermediarias.clienteFoPag',[0,1])
massa_SegmentacaoPoliticaAntigos.gera_cenario('payloadHomol.intermediarias.principalidadeAll',[0,1])
massa_SegmentacaoPoliticaAntigos.gera_cenario('solicitante.flagMAT',[0,1])
massa_SegmentacaoPoliticaAntigos.gera_cenario('solicitante.rendaLiquidaPicpay',[-99999,0,3442.36,3442.37,3442.38])

massa_SegmentacaoPoliticaAntigos.get_massa_final()

massa_SegmentacaoPoliticaAntigos.set_bom_arqv()
massa_SegmentacaoPoliticaAntigos.converte_massa()

### Execução Modular e Geração de CSV

In [ ]:
listaResultado = rp.execucaoModular(massa_SegmentacaoPoliticaAntigos.get_massa_json(),rp.SegmentacaoPoliticaAntigos)

##### Valida se todas condições estão sendo abordadas

In [ ]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['segmentacaoPolitica'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

In [ ]:
# Renomeação das colunas para o cabeçalho do RMA
massa_SegmentacaoPoliticaAntigos.rename_cabecalho()

In [ ]:
massa_SegmentacaoPoliticaAntigos.get_massa_dm()

In [ ]:
massa_SegmentacaoPoliticaAntigos.atribui_expected(listaResultado,"payloadHomol.intermediarias.segmentacaoPolitica","segmentacaoPolitica")
massa_SegmentacaoPoliticaAntigos.get_massa_dm()

### Conversão da massa para CSV

In [ ]:
massa_SegmentacaoPoliticaAntigos.get_massa_dm().to_csv('C:/Users/T51657/Desktop/Homologação/Modelos Online/v2/massas/Modulares/massaSegmentacaoPoliticaAntigos.csv',index=False)